In [1]:
!pip install gradio
import gradio as gr
import pandas as pd
import os
!pip install ultralytics
from ultralytics import YOLO
from PIL import Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.3 which is 

In [2]:
import gradio as gr
import pandas as pd
import os
from ultralytics import YOLO
from PIL import Image

# Load a pre-trained YOLO model (you can specify a custom YOLO model here)
model = YOLO("yolov5s.pt")  # Using YOLOv5 small pre-trained model

# Function to run YOLO detection on a list of images
def detect_objects_in_files(files):
    results_list = []
    
    for file in files:
        img = Image.open(file.name)

        # Run YOLO detection
        results = model(img)

        # Parse the results
        for r in results:
            boxes = r.boxes.xyxy.cpu().numpy()  # Bounding boxes
            scores = r.boxes.conf.cpu().numpy()  # Confidence scores
            classes = r.boxes.cls.cpu().numpy()  # Class labels
            for box, score, cls in zip(boxes, scores, classes):
                results_list.append({
                    'image': os.path.basename(file.name),
                    'class': int(cls),
                    'confidence': float(score),
                    'x_min': box[0],
                    'y_min': box[1],
                    'x_max': box[2],
                    'y_max': box[3]
                })
    
    # Convert results into a dataframe
    df = pd.DataFrame(results_list)
    return df

# Gradio interface
def process_files(files):
    # Process the list of uploaded image files and return dataframe results
    df = detect_objects_in_files(files)
    return df

# Create Gradio app
input_files = gr.File(file_count="multiple", label="Upload images (multiple files allowed)")
output_df = gr.Dataframe(label="Detection Results", headers=["image", "class", "confidence", "x_min", "y_min", "x_max", "y_max"])

# Launch Gradio interface
gr.Interface(fn=process_files, 
             inputs=input_files, 
             outputs=output_df, 
             title="YOLO Object Detection on Images").launch()


PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 43.9MB/s]


Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://d29fe9773e3ca1668f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [3]:
# help(gr)